In [67]:
import os
import sys
import glob
import cv2
import matplotlib.pyplot as plt # 주피터 환경에서 이미지 열어주는거
import numpy as np

STUDENT_CODE = '2017147505'
FILE_NAME = 'output.txt'
if not os.path.exists(STUDENT_CODE):
    os.mkdir(STUDENT_CODE)
f = open(os.path.join(STUDENT_CODE, FILE_NAME),'w')

# f.write("aaa")
# f.close()
# 이미지를 전부 읽어와서 vectorize한 다음 하나로 합친다.
train_images = []

image = cv2.imread("faces_training/face01.pgm", cv2.IMREAD_GRAYSCALE)
train_images.append(image)

length, height = image.shape
image = image.reshape((length * height * 1, 1))

for i in range(2, 10):
    x = cv2.imread(f"faces_training/face0{i}.pgm", cv2.IMREAD_GRAYSCALE)
    train_images.append(x)
    temp = x.reshape((length * height * 1, 1))
    image = np.concatenate([image, temp], axis=1)

for i in range(10, 40):
    x = cv2.imread(f"faces_training/face{i}.pgm", cv2.IMREAD_GRAYSCALE)
    train_images.append(x)
    temp = x.reshape((length * height * 1, 1))
    # print(temp.shape)
    image = np.concatenate([image, temp], axis=1)

train_all_image = image
print(image.shape)


def computeDimensions(img, percent=0.9):
    
    # computing eigenvalues and eigenvectors of covariance matrix
    img_mean = np.mean(img.T, axis=1)
    img = (img - img_mean)

    U, Sv, Vt = np.linalg.svd(img, full_matrices=False)
    
    whole_nom_of_pc = np.size(U, axis=0)
    
    # Sv 는 고유값 리스트이다. 
    Sv = sorted(Sv)
    Sv = Sv[::-1]  # 순서를 높은게 앞에 오도록 변경
    eigvals = [x**2 for x in Sv]
    
    sum_of = 0
    numPc=0
    sum_all = sum(eigvals)
    
    for e in eigvals:
        sum_of += e
        numPc += 1
        if(sum_of/sum_all >= percent):
            break
    
    U = U[:, :numPc]  # 입력된 pc 개수에 따라 선택한다.
    Sv = np.diag(Sv[:numPc])
    Vt = Vt[:numPc, :]
    reconstruct = np.dot(U, np.dot(Sv,Vt)) + img_mean
    
    return reconstruct, numPc

# perc = sys.argv[1]
perc = 0.95

reconstruct, numPc = computeDimensions(image, perc)


print("######### Step1 #########")
f.write("######### Step1 #########\n")
print(f"Input Percentage: {perc}")
f.write(f"Input Percentage: {perc}\n")
print(f"Selected Dimansion : {numPc}")
f.write(f"Selected Dimansion : {numPc}\n")


print(reconstruct.shape)

(32256, 39)
######### Step1 #########
Input Percentage: 0.95
Selected Dimansion : 25
(32256, 39)


In [68]:
import matplotlib.pyplot as plt # 주피터 환경에서 이미지 열어주는거

f = open(os.path.join(STUDENT_CODE, FILE_NAME),'w')

print("######### Step2 #########")
f.write("######### Step2 #########\n")

print("Reconstruction error")
f.write("Reconstruction error\n")

re_error = []
re_imgs = []

for n in range(39):
    re_err_sum = 0
    re_img = reconstruct.T[n].reshape(length, height)
    for i in range(length):
        for j in range(height):
            re_err_sum += np.abs(train_images[n][i][j] - re_img[i][j])**2
    
    re_error.append(re_err_sum/(length*height))
    re_imgs.append(re_img)  

print(f"average : {sum(re_error)/len(re_error)}")
f.write(f"average : {sum(re_error)/len(re_error)}\n")
for n in range(1, 10):
    print(f"0{n}: {re_error[n-1]}")
    f.write(f"0{n}: {re_error[n-1]}\n")
    cv2.imwrite(f"{STUDENT_CODE}/face0{n}.png", re_imgs[n-1])
    
for n in range(10, 40):
    print(f"{n}: {re_error[n-1]}")
    f.write(f"{n}: {re_error[n-1]}\n")
    cv2.imwrite(f"{STUDENT_CODE}/face{n}.png", re_imgs[n-1])

f.close()

######### Step2 #########
Reconstruction error
average : 80.743188108986
01: 90.10954779754424
02: 81.60834060202127
03: 112.8817515195717
04: 144.74993788304403
05: 49.93015541452326
06: 88.28189025720393
07: 122.9529913987947
08: 158.62993364811305
09: 128.27324084545663
10: 86.71980615317213
11: 85.94446982122062
12: 47.252939530553185
13: 40.75161324518188
14: 4.153125911714489
15: 41.552103159414756
16: 62.2663483728361
17: 35.34815073430032
18: 58.54195654920177
19: 77.53021033215252
20: 146.88566848663712
21: 23.11861013826295
22: 11.707684612456228
23: 96.85332580659627
24: 96.67877166806427
25: 69.05642876200493
26: 53.41433113497292
27: 11.515978868467514
28: 57.228583270679735
29: 26.99078555663959
30: 109.96545113832336
31: 165.62928643846487
32: 88.12391283583099
33: 131.15586191970414
34: 33.49517468118154
35: 147.0249020452186
36: 81.22056645787144
37: 114.7037513371943
38: 141.41068382243375
39: 25.326064093429256


In [70]:
# reconstruct

print("######### Step3 #########")

test_images = []

imagett = cv2.imread("faces_test/test01.pgm", cv2.IMREAD_GRAYSCALE)
test_images.append(image)

img_size = imagett.shape
imagett = imagett.reshape((length * height * 1, 1))

for i in range(2, 6):
    x = cv2.imread(f"faces_test/test0{i}.pgm", cv2.IMREAD_GRAYSCALE)
    test_images.append(x)
    temp = x.reshape((length * height * 1, 1))
    imagett = np.concatenate([imagett, temp], axis=1)

reconstruct_mean = np.mean(reconstruct.T, axis=1)
reconstruct_reduc_mean = reconstruct - reconstruct_mean

U, S, Vh = np.linalg.svd(reconstruct_reduc_mean, full_matrices=False)
U = U[:,:numPc]

face_proj = U.T @ reconstruct
test_proj = U.T @ imagett

def similarity_between_faces(face1, face2, plot=False):
    face1 -= 1
    face2 -= 1
    if plot:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,16))
        ax1.imshow(imagett[:, face1].reshape(img_size), cmap='gray')
        ax2.imshow(reconstruct[:, face2].reshape(img_size), cmap='gray')
    
    face_diff = test_proj[:, face1] - face_proj[:, face2]
    
    return np.linalg.norm(face_diff)

# plot 3 most similar faces
def find_most_similar(face, plot=False):
    best_scores = [9999999, 9999999, 9999999]
    best_face_i = [0, 0, 0]

    distances = []
    for face_i in range(39):
        sim = similarity_between_faces(face, face_i)
        distances.append(sim)
    
    best_scores[0] = min(distances)
    best_face_i[0] = distances.index(min(distances))
    distances[best_face_i[0]] = 999999
    
    if plot == True:
        plt.imshow(imagett[:, face-1].reshape(img_size), cmap='gray')
        plt.title('Compare to this')
        plt.show()

        plt.imshow(train_all_image[:, best_face_i[0]-1].reshape(img_size), cmap='gray')
        plt.title("Score: {}".format(int(best_scores[0])))
        plt.show()
    
    return best_face_i[0]


for a in range(1,6):
    found = find_most_similar(a, plot=False)
    
    if found >= 10 : 
        print(f"test0{a}.pgm ==> face{found}.pgm")
    else : 
        print(f"test0{a}.pgm ==> face0{found}.pgm")


######### Step3 #########
(25, 39)
(25, 5)
test01.pgm ==> face03.pgm
test02.pgm ==> face07.pgm
test03.pgm ==> face19.pgm
test04.pgm ==> face23.pgm
test05.pgm ==> face33.pgm


0.0


In [76]:
import os
import sys
import glob
import cv2
import matplotlib.pyplot as plt # 주피터 환경에서 이미지 열어주는거
import numpy as np

STUDENT_CODE = '2017147505'
FILE_NAME = 'output.txt'
if not os.path.exists(STUDENT_CODE):
    os.mkdir(STUDENT_CODE)
f = open(os.path.join(STUDENT_CODE, FILE_NAME),'w')

# f.write("aaa")
# f.close()
# 이미지를 전부 읽어와서 vectorize한 다음 하나로 합친다.
train_images = []

image = cv2.imread("faces_training/face01.pgm", cv2.IMREAD_GRAYSCALE)
train_images.append(image)

length, height = image.shape
image = image.reshape((length * height * 1, 1))

for i in range(2, 10):
    x = cv2.imread(f"faces_training/face0{i}.pgm", cv2.IMREAD_GRAYSCALE)
    train_images.append(x)
    temp = x.reshape((length * height * 1, 1))
    image = np.concatenate([image, temp], axis=1)

for i in range(10, 40):
    x = cv2.imread(f"faces_training/face{i}.pgm", cv2.IMREAD_GRAYSCALE)
    train_images.append(x)
    temp = x.reshape((length * height * 1, 1))
    image = np.concatenate([image, temp], axis=1)

train_all_image = image


def computeDimensions(img, percent=0.9):
    
    # computing eigenvalues and eigenvectors of covariance matrix
    img_mean = np.mean(img.T, axis=1)
    img = (img - img_mean)

    U, Sv, Vt = np.linalg.svd(img, full_matrices=False)
    
    whole_nom_of_pc = np.size(U, axis=0)
    
    # Sv 는 고유값 리스트이다. 
    Sv = sorted(Sv)
    Sv = Sv[::-1]  # 순서를 높은게 앞에 오도록 변경
    eigvals = [x**2 for x in Sv]
    
    sum_of = 0
    numPc=0
    sum_all = sum(eigvals)
    
    for e in eigvals:
        sum_of += e
        numPc += 1
        if(sum_of/sum_all >= percent):
            break
    
    U = U[:, :numPc]  # 입력된 pc 개수에 따라 선택한다.
    Sv = np.diag(Sv[:numPc])
    Vt = Vt[:numPc, :]
    reconstruct = np.dot(U, np.dot(Sv,Vt)) + img_mean
    
    return reconstruct, numPc

# perc = sys.argv[1]
perc = 0.95

reconstruct, numPc = computeDimensions(image, perc)


f.write("######### STEP 1 #########\n")
print(f"Input Percentage: {perc}")
f.write(f"Input Percentage: {perc}\n")
print(f"Selected Dimansion : {numPc}")
f.write(f"Selected Dimansion : {numPc}\n")


f.write("\n######### STEP 2 #########\n")

print("Reconstruction error")
f.write("Reconstruction error\n")

re_error = []
re_imgs = []

for n in range(39):
    re_err_sum = 0
    re_img = reconstruct.T[n].reshape(length, height)
    for i in range(length):
        for j in range(height):
            re_err_sum += np.abs(train_images[n][i][j] - re_img[i][j])**2
    
    re_error.append(re_err_sum/(length*height))
    re_imgs.append(re_img)  

print(f"average : {sum(re_error)/len(re_error)}")
f.write(f"average : {sum(re_error)/len(re_error)}\n")
for n in range(1, 10):
    print(f"0{n}: {re_error[n-1]}")
    f.write(f"0{n}: {re_error[n-1]}\n")
    cv2.imwrite(f"{STUDENT_CODE}/face0{n}.png", re_imgs[n-1])
    
for n in range(10, 40):
    print(f"{n}: {re_error[n-1]}")
    f.write(f"{n}: {re_error[n-1]}\n")
    cv2.imwrite(f"{STUDENT_CODE}/face{n}.png", re_imgs[n-1])

f.write("\n########## STEP 3 ##########")

test_images = []

imagett = cv2.imread("faces_test/test01.pgm", cv2.IMREAD_GRAYSCALE)
test_images.append(image)

img_size = imagett.shape
imagett = imagett.reshape((length * height * 1, 1))

for i in range(2, 6):
    x = cv2.imread(f"faces_test/test0{i}.pgm", cv2.IMREAD_GRAYSCALE)
    test_images.append(x)
    temp = x.reshape((length * height * 1, 1))
    imagett = np.concatenate([imagett, temp], axis=1)

reconstruct_mean = np.mean(reconstruct.T, axis=1)
reconstruct_reduc_mean = reconstruct - reconstruct_mean

U, S, Vh = np.linalg.svd(reconstruct_reduc_mean, full_matrices=False)
U = U[:,:numPc]

face_proj = U.T @ reconstruct
test_proj = U.T @ imagett

def similarity_between_faces(face1, face2, plot=False):
    face1 -= 1
    face2 -= 1
    if plot:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,16))
        ax1.imshow(imagett[:, face1].reshape(img_size), cmap='gray')
        ax2.imshow(reconstruct[:, face2].reshape(img_size), cmap='gray')
    
    face_diff = test_proj[:, face1] - face_proj[:, face2]
    
    return np.linalg.norm(face_diff)

# plot 3 most similar faces
def find_most_similar(face, plot=False):
    best_scores = [9999999, 9999999, 9999999]
    best_face_i = [0, 0, 0]

    distances = []
    for face_i in range(39):
        sim = similarity_between_faces(face, face_i)
        distances.append(sim)
    
    best_scores[0] = min(distances)
    best_face_i[0] = distances.index(min(distances))
    distances[best_face_i[0]] = 999999
    
    if plot == True:
        plt.imshow(imagett[:, face-1].reshape(img_size), cmap='gray')
        plt.title('Compare to this')
        plt.show()

        plt.imshow(train_all_image[:, best_face_i[0]-1].reshape(img_size), cmap='gray')
        plt.title("Score: {}".format(int(best_scores[0])))
        plt.show()
    
    return best_face_i[0]


for a in range(1,6):
    found = find_most_similar(a, plot=False)
    
    if found >= 10 : 
        print(f"\ntest0{a}.pgm ==> face{found}.pgm")
        f.write(f"\ntest0{a}.pgm ==> face{found}.pgm")
    else : 
        print(f"\ntest0{a}.pgm ==> face0{found}.pgm")
        f.write(f"\ntest0{a}.pgm ==> face0{found}.pgm")

f.close()


######### STEP1 #########
Input Percentage: 0.95
Selected Dimansion : 25

######### STEP2 #########
Reconstruction error
average : 80.743188108986
01: 90.10954779754424
02: 81.60834060202127
03: 112.8817515195717
04: 144.74993788304403
05: 49.93015541452326
06: 88.28189025720393
07: 122.9529913987947
08: 158.62993364811305
09: 128.27324084545663
10: 86.71980615317213
11: 85.94446982122062
12: 47.252939530553185
13: 40.75161324518188
14: 4.153125911714489
15: 41.552103159414756
16: 62.2663483728361
17: 35.34815073430032
18: 58.54195654920177
19: 77.53021033215252
20: 146.88566848663712
21: 23.11861013826295
22: 11.707684612456228
23: 96.85332580659627
24: 96.67877166806427
25: 69.05642876200493
26: 53.41433113497292
27: 11.515978868467514
28: 57.228583270679735
29: 26.99078555663959
30: 109.96545113832336
31: 165.62928643846487
32: 88.12391283583099
33: 131.15586191970414
34: 33.49517468118154
35: 147.0249020452186
36: 81.22056645787144
37: 114.7037513371943
38: 141.41068382243375
39: 2